# 02 Trend Scanning


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [2]:
from src.signals.trend_scanning import trend_scanning

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

scan = trend_scanning(
    prices=prices[themes[0]],
    lookback_min=cfg["trend_scanning"]["lookback_min"],
    lookback_max=cfg["trend_scanning"]["lookback_max"],
    step=cfg["trend_scanning"]["step"],
    use_log=cfg["trend_scanning"]["use_log"],
)

assert scan.shape[0] == len(dates)
assert set(["tval", "horizon", "slope", "label", "t1"]).issubset(scan.columns)

save_parquet(scan, PROJECT_ROOT / "outputs/signals/02_trend_scanning.parquet")
save_csv(scan.tail(300), PROJECT_ROOT / "outputs/diagnostics/02_trend_scanning_sample.csv")

fig, ax1 = plt.subplots(figsize=(10, 4))
scan["tval"].plot(ax=ax1, color="tab:blue", alpha=0.8)
ax1.set_ylabel("t-value", color="tab:blue")
ax1.set_title("02 Trend Scanning (sample)")
ax2 = ax1.twinx()
scan["horizon"].plot(ax=ax2, color="tab:orange", alpha=0.4)
ax2.set_ylabel("horizon", color="tab:orange")
save_figure(fig, PROJECT_ROOT / "outputs/figures/02_trend_scan_tval_horizon.png")
plt.close(fig)

print(scan[["tval", "horizon", "slope"]].tail())


                 tval  horizon     slope
date                                    
2025-12-25 -29.028303    120.0 -0.001613
2025-12-26 -29.419351    120.0 -0.001639
2025-12-29 -30.145663    120.0 -0.001667
2025-12-30 -30.707737    120.0 -0.001697
2025-12-31 -30.907763    120.0 -0.001714


In [3]:
required = [
    "outputs/signals/02_trend_scanning.parquet",
    "outputs/diagnostics/02_trend_scanning_sample.csv",
    "outputs/figures/02_trend_scan_tval_horizon.png",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=02_trend_scanning.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)


Notebook=02_trend_scanning.ipynb, OK, 生成ファイル確認=outputs/signals/02_trend_scanning.parquet:True, outputs/diagnostics/02_trend_scanning_sample.csv:True, outputs/figures/02_trend_scan_tval_horizon.png:True

